<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ie2t3YaOHiPyuxpgb6DjchE_lCPnCJwN?usp=sharing)


## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- Join Innovation Community

Learn by building. Get expert mentorship and work on real AI projects.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)

##Step-by-Step Guide to Building AI Agent Teams with CrewAI


###Introduction
CrewAI is a powerful framework that allows you to create, coordinate and deploy teams of AI agents that work together to accomplish complex tasks. This guide will walk you through the process of creating your own AI agent teams, even if you have minimal coding experience.

###install Required Libraries

In [ ]:
!pip install -qU crewai langchain_openai 'crewai[tools]' google-search-results

###Getting Your API Keys
**Serper API Key**

* Visit https://serper.dev/api-key
* Sign up for an account or log in
* Navigate to the API dashboard
* Generate your API key
* Copy the key and use in your Colab Notebook

**Google Gemini API Key**

* Visit https://aistudio.google.com/
 Sign in with your Google account
* Navigate to "API Keys" in the settings menu
* Click "Create API Key"
* Copy your new API key And use in Colab Notebook


In [ ]:
import os
from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

###Initialize Models

In [ ]:
from langchain_openai import ChatOpenAI
from crewai import LLM

Gemini = LLM(
    model="gemini/gemini-2.0-flash",)


#gpt4o = ChatOpenAI(model = "gpt-4o")

### AI Customer Support Agent ðŸ§³


In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI
from crewai import LLM

# Initialize tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
tools = [search_tool, scrape_tool]

def create_customer_support_system(customer_query, product_name, knowledge_base_url):
    """
    Creates and runs a customer support system with multiple specialized agents
    to handle customer inquiries.

    Args:
        customer_query (str): The customer's support request or question
        product_name (str): Name of the product customer is inquiring about
        knowledge_base_url (str): URL to the product's knowledge base/documentation

    Returns:
        str: A comprehensive support response
    """
    # Define the Query Classifier Agent
    classifier_agent = Agent(
        role="Support Query Classifier",
        goal="Accurately classify customer support queries by type and priority",
        backstory="An expert in customer support triage with the ability to identify query type, urgency, and required expertise.",
        verbose=True,
        tools=tools,
        llm=Gemini
    )

    # Define the Technical Support Agent
    technical_agent = Agent(
        role="Technical Support Specialist",
        goal=f"Provide accurate technical solutions for {product_name} issues",
        backstory=f"A senior engineer with deep knowledge of {product_name}'s architecture and common technical issues. Known for explaining complex solutions in simple terms.",
        verbose=True,
        tools=tools,
        llm=Gemini
    )

    # Define the Customer Relations Agent
    customer_agent = Agent(
        role="Customer Relations Manager",
        goal="Ensure customer satisfaction and provide empathetic communication",
        backstory="A customer service expert specialized in creating positive customer experiences, with expertise in communication psychology and conflict resolution.",
        verbose=True,
        tools=tools,
        llm=Gemini
    )

    # Define the Tasks

    # Task 1: Classify the query
    classification_task = Task(
        description=f"Analyze this customer query: '{customer_query}' about product '{product_name}'. " +
                   "Classify it by type (technical issue, billing question, feature request, etc.), " +
                   "determine priority level (low, medium, high, urgent), and identify what expertise is needed to resolve it.",
        expected_output="A detailed classification with query type, priority level, required expertise, and recommended next steps.",
        agent=classifier_agent
    )

    # Task 2: Technical solution research
    technical_task = Task(
        description=f"Research and provide a solution for this query: '{customer_query}' about '{product_name}'. " +
                   f"Use the knowledge base at {knowledge_base_url} and search for relevant documentation. " +
                   "Provide step-by-step troubleshooting instructions with screenshots or diagrams if helpful.",
        expected_output="A comprehensive technical solution with step-by-step instructions that completely resolves the customer's issue.",
        agent=technical_agent
    )

    # Task 3: Craft the final response
    response_task = Task(
        description=f"Using the technical solution provided, craft a friendly, empathetic response to '{customer_query}'. " +
                  "The response should be professional but warm, easy to understand, and should anticipate follow-up questions. " +
                  "Include appropriate reassurances and clear next steps if the issue requires additional support.",
        expected_output="A complete customer-ready response that balances technical accuracy with customer care principles.",
        agent=customer_agent
    )

    # Create and Run the Crew
    support_crew = Crew(
        agents=[classifier_agent, technical_agent, customer_agent],
        tasks=[classification_task, technical_task, response_task],
        verbose=True,
        process=Process.sequential
    )

    result = support_crew.kickoff(
        inputs={
            "customer_query": customer_query,
            "product_name": product_name,
            "knowledge_base_url": knowledge_base_url
        }
    )

    return result

### Execute the agent


In [ ]:
print("AI Customer Support System")
print("==========================")

product_name = input("Enter product name: ")
knowledge_base_url = input("Enter knowledge base URL: ")
print("\nWhat issue is the customer experiencing?")
customer_query = input("> ")

print("\nProcessing support request...\n")
support_response = create_customer_support_system(customer_query, product_name, knowledge_base_url)

print("\nCustomer Support Response:")
print("==========================")
print(support_response)